In [4]:
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import time
from selenium.webdriver.common.action_chains import ActionChains


In [5]:
url  = 'https://www.goodreads.com/list/show/55.Disappointing_Books?page='
link = 'https://www.goodreads.com/'


In [6]:
binary = FirefoxBinary('C:/Program Files/Mozilla Firefox/firefox.exe')
driver = webdriver.Firefox(firefox_binary=binary)


In [7]:

def scroll_shim(passed_in_driver, object):
        x = object.location['x']
        y = object.location['y']
        scroll_by_coord = 'window.scrollTo(%s,%s);' % (
            x,
            y
        )
        scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
        passed_in_driver.execute_script(scroll_by_coord)
        passed_in_driver.execute_script(scroll_nav_out_of_way)
 

In [ ]:
for i in range(5,7):
    url  = 'https://www.goodreads.com/list/show/55.Disappointing_Books?page='
    url     = url +str(i)
    soup    = BeautifulSoup(requests.get(url).text,features="lxml")
    request = requests.get(url)
    
    BeautifulSoup(request.text, 'html.parser')  
    for a in soup.select('a[class="bookTitle"]'):
        
        url  = link+a['href']
        print(url)
        
        soup = BeautifulSoup(requests.get(url).text,features="lxml")
        soup.findAll("body")

        request  =  requests.get(url)
        BeautifulSoup(request.text, 'html.parser')
                

        Title   = soup.select('h1[id="bookTitle"]')
        Title   = [re.sub(r'<.+?>',r'',str(a)) for a in Title]
        Title   = [re.sub(r'\n',r'',str(a)) for a in Title]
        b = "!@/;+-*()[]_{}:#$',\#?éà@\""
        Title   = str(Title)
        for char in b:
            Title = Title.replace(char,"")      
        Title     = str(Title).translate({ord(i): None for i in "[':]"})
        Title     = Title.strip(':')
        
        Comments  = soup.select('span[id^="reviewTextContainer"]')
        Comments  = [re.sub(r'<.+?>',r'',str(a)) for a in Comments]
        Comments  = [re.sub(r'\n',r'',str(a)) for a in Comments]
        
        #Driver
        driver.get(url) 
        html = driver.page_source
        
        time.sleep(2)
        liste = []
        for i in range(1,5):
            element = driver.find_element_by_xpath("//*[@id='reviews']/div[5]/div/a["+str(i)+"]")
            
            time.sleep(3)
            if 'firefox' in driver.capabilities['browserName']:
                scroll_shim(driver, element)
            actions = ActionChains(driver)
            actions.move_to_element(element)
            actions.click()
            actions.perform()
            time.sleep(10)
            all_comments  =  driver.find_elements_by_xpath("//*[contains(@id,'freeTextContainer')]")
            
            for c in all_comments:
                time.sleep(2)
                liste.insert(len(liste),c.text)
                print(len(liste))
                driver.refresh
            time.sleep(2)
            
        liste          = pd.DataFrame(liste)
        df_Comments    = pd.DataFrame(Comments)
        #final_result   = pd.concat([df_Comments,liste])
        
        df        = pd.DataFrame(liste)
        df2       = pd.DataFrame(df_Comments)
        frames    = [df2,df]
        result    = pd.concat(frames)
        result.rename(columns={ df.columns[0]: "Comments" })
        result.to_csv('E:/BD2C/Semestre III/Text minning/Projet/Dataset2/'+str(Title)+'.csv', header=['Comments']   , index=None, sep=' ', mode='a')
